# Optimización de la Selección de Pozos de Petróleo

Trabajas para la compañía minera OilyGiant. Tu tarea es encontrar el mejor lugar para un nuevo pozo.
Pasos para elegir la ubicación:

- Recolecta los parámetros del pozo de petróleo en la región seleccionada: calidad del petróleo y volumen de reservas
- Construye un modelo para predecir el volumen de reservas en los nuevos pozos
- Selecciona los pozos de petróleo con los valores estimados más altos
- Elige la región con el mayor beneficio total para los pozos de petróleo seleccionados

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

# Descripción de los datos

- `id` — Identificador único de pozo de petróleo
- `f0, f1, f2` — Tres características sobre la calidad del petróleo (su significado específico no es importante, pero las características en sí son significativas)
- `product` — Volumen de reservas en el pozo de petróleo (miles de barriles)

**Condiciones**:

- Solo la regresión lineal es adecuada para el entrenamiento de modelos.
- Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
- El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
- Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).
- Después de la evaluación de riesgo, mantén solo las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.

# Inicialización

In [511]:
# Carga todas las librerías
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Cargar los datos

In [512]:
# Carga los datos en DataFrames
try:
    df0 = pd.read_csv('/datasets/geo_data_0.csv')
    df1 = pd.read_csv('/datasets/geo_data_1.csv')
    df2 = pd.read_csv('/datasets/geo_data_2.csv')
except Exception:
    df0 = pd.read_csv('datasets/geo_data_0.csv')
    df1 = pd.read_csv('datasets/geo_data_1.csv')
    df2 = pd.read_csv('datasets/geo_data_2.csv')

# Preparar los datos

In [513]:
# Imprime la información general/resumen sobre el DataFrame
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [514]:
# Imprime una muestra aleatoria de 5 filas del DataFrame
df0.sample(5)

,id,f0,f1,f2,product
90354,OZtA2,1.234302,-0.328379,0.111507,54.934129
69593,CUHW7,1.254042,-0.429553,-6.808379,84.776590
45153,BbX27,0.507428,0.903823,0.137340,40.544101
65496,TWMOr,-0.150369,1.001128,3.405462,85.233130
87801,njEhr,0.485322,0.945299,0.793361,49.338800


In [515]:
# Imprime la información general/resumen sobre el DataFrame
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [516]:
# Imprime una muestra aleatoria de 5 filas del DataFrame
df1.sample(5)

,id,f0,f1,f2,product
92559,0xBA2,-5.376025,-1.393992,3.003793,84.038886
77545,X1fdQ,-6.596175,-2.399353,1.000795,30.132364
10800,Traxs,8.746752,-0.100752,1.006119,26.953261
14735,9JdSU,11.158535,-1.777187,1.997798,53.906522
45120,jkksI,5.137751,-0.525132,0.997621,26.953261


In [517]:
# Imprime la información general/resumen sobre el DataFrame
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [518]:
# Imprime una muestra aleatoria de 5 filas del DataFrame
df2.sample(5)

,id,f0,f1,f2,product
81891,V77tl,-4.884031,0.587705,5.722631,176.162360
34479,s0UYW,1.658289,-0.027524,0.488845,49.046223
50289,iGshN,-0.918801,0.084935,2.577698,22.463836
56811,lc21C,0.882288,-0.867702,5.263757,109.557760
76962,tsegU,0.854186,-0.006871,6.363004,117.068729


## Observaciones
- La columna `id` no contiene información relevante para las predicciones del modelo.
- Ninguno de los tres datasets contiene valores ausentes, así como el tipo de dato de todas las columnas es correcto.
- Los tres datasets están completamente listos para ser utilizados en el entrenamiento de modelos.

## Eliminar columnas innecesarias

In [519]:
# Elimina las columnas que no se utilizarán en el análisis
df0 = df0.drop(['id'], axis=1)
df1 = df1.drop(['id'], axis=1)
df2 = df2.drop(['id'], axis=1)

# Segmentar los datos

In [520]:
# Crear un estado aleatorio a partir de un seed (semilla) fijo
random_state = np.random.RandomState(12345)

## Region 0

In [521]:
# Divide el DataFrame 0 en características (features) y objetivo (target)
features0 = df0.drop(['product'], axis=1)
target0 = df0['product']

In [522]:
# Separar los datos en subconjuntos de entrenamiento y validación en una proporción de 75:25
features_train0, features_valid0, target_train0, target_valid0 = train_test_split(features0, target0, test_size=0.25, random_state=random_state)

## Region 1

In [523]:
# Divide el DataFrame 1 en características (features) y objetivo (target)
features1 = df1.drop(['product'], axis=1)
target1 = df1['product']

In [524]:
# Separar los datos en subconjuntos de entrenamiento y validación en una proporción de 75:25
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(features1, target1, test_size=0.25, random_state=random_state)

## Region 2

In [525]:
# Divide el DataFrame 2 en características (features) y objetivo (target)
features2 = df2.drop(['product'], axis=1)
target2 = df2['product']

In [526]:
# Separar los datos en subconjuntos de entrenamiento y validación en una proporción de 75:25
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(features2, target2, test_size=0.25, random_state=random_state)

# Entrenamiento del modelo (Regresión lineal)

## Region 0

In [527]:
# Probar el modelo de Regresión Lineal en el conjunto de datos 0
model0 = LinearRegression()
model0.fit(features_train0, target_train0)
predicted_valid0 = model0.predict(features_valid0)
rmse0 = mean_squared_error(target_valid0, predicted_valid0) ** 0.5

# Guardar las predicciones y las respuestas correctas para el conjunto de validación
predictions0 = pd.Series(predicted_valid0, index=target_valid0.index)

# Mostrar en pantalla el volumen promedio de las reservas previstas y la RECM del modelo
print('Volumen promedio de las reservas previstas para la región 0:', predicted_valid0.mean())
print('RECM para la región 0:', rmse0)

Volumen promedio de las reservas previstas para la región 0: 92.59256778438035
RECM para la región 0: 37.5794217150813


### Observaciones
- El volumen promedio de las reservas previstas para la **`región 0`** es de `92.59 miles de barriles`.
- El RMSE para la **`región 0`** es de `37.58`.

## Region 1

In [528]:
# Probar el modelo de Regresión Lineal en el conjunto de datos 1
model1 = LinearRegression()
model1.fit(features_train1, target_train1)
predicted_valid1 = model1.predict(features_valid1)
rmse1 = mean_squared_error(target_valid1, predicted_valid1) ** 0.5

# Guardar las predicciones y las respuestas correctas para el conjunto de validación
predictions1 = pd.Series(predicted_valid1, index=target_valid1.index)

# Mostrar en pantalla el volumen promedio de las reservas previstas y la RECM del modelo
print('Volumen promedio de las reservas previstas para la región 1:', predicted_valid1.mean())
print('RECM para la región 1:', rmse1)

Volumen promedio de las reservas previstas para la región 1: 68.76995145799752
RECM para la región 1: 0.8897367737680646


### Observaciones
- El volumen promedio de las reservas previstas para la **`región 1`** es de `68.77 miles de barriles`.
- El RMSE para la **`región 1`** es de `0.89`.

## Region 2

In [529]:
# Probar el modelo de Regresión Lineal en el conjunto de datos 2
model2 = LinearRegression()
model2.fit(features_train2, target_train2)
predicted_valid2 = model2.predict(features_valid2)
rmse2 = mean_squared_error(target_valid2, predicted_valid2) ** 0.5

# Guardar las predicciones y las respuestas correctas para el conjunto de validación
predictions2 = pd.Series(predicted_valid2, index=target_valid2.index)

# Mostrar en pantalla el volumen promedio de las reservas previstas y la RECM del modelo
print('Volumen promedio de las reservas previstas para la región 2:', predicted_valid2.mean())
print('RECM para la región 2:', rmse2)

Volumen promedio de las reservas previstas para la región 2: 95.087528122523
RECM para la región 2: 39.958042459521614


### Observaciones
- El volumen promedio de las reservas previstas para la **`región 2`** es de `95.09 miles de barriles`.
- El RECM para la **`región 2`** es de `39.96`.

## Conclusion intermedia
-  Las reservas promedio más altas se encuentran en la `región 2`, seguidas de cerca por la `región 0`. Sin embargo, es notable que el **RECM** es considerablemente alto en la `región 0 y 2`, lo que indica un nivel más alto de error en las predicciones de las reservas de petróleo en estas regiones. En contraste, la `región 1` tiene una reserva promedio más baja, pero también tiene un **RECM** mucho más bajo, lo que sugiere que las predicciones del modelo son más precisas en esta región.

# Cálculo de ganancias

## Valores clave para los cálculos

In [530]:
# Presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
budget = 100_000_000

# El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).
income_per_unit = 4_500

# Riesgo de pérdidas inferior al 2.5%
risk_threshold = 0.025

# Coste de desarrollo de un solo pozo
cost_per_well = budget / 200
print('Coste de desarrollo de un solo pozo:', cost_per_well)

# Volumen de reservas suficiente para desarrollar un nuevo pozo sin pérdidas
sufficient_volume = cost_per_well / income_per_unit
print('Volumen de reservas suficiente para desarrollar un nuevo pozo sin pérdidas:', sufficient_volume)

Coste de desarrollo de un solo pozo: 500000.0
Volumen de reservas suficiente para desarrollar un nuevo pozo sin pérdidas: 111.11111111111111


### Observaciones
- Coste de desarrollo de un solo pozo: `500,000`
- Volumen de reservas suficiente para desarrollar un nuevo pozo sin pérdidas: `111.11`

## Volumen de reservas suficiente para desarrollar un nuevo pozo sin pérdidas

### Region 0

In [531]:
print('Volumen medio de reservas para la región 0:', target0.mean())

# Compara el valor obtenido con el volumen medio de reservas para la región 0
if sufficient_volume > target0.mean():
    print('No es posible desarrollar un nuevo pozo en la región 0 sin pérdidas.')
else:
    print('Es posible desarrollar un nuevo pozo en la región 0 sin pérdidas.')

Volumen medio de reservas para la región 0: 92.50000000000001
No es posible desarrollar un nuevo pozo en la región 0 sin pérdidas.


### Region 1

In [532]:
print('Volumen medio de reservas para la región 1:', target1.mean())

# Compara el valor obtenido con el volumen medio de reservas para la región 1
if sufficient_volume > target1.mean():
    print('No es posible desarrollar un nuevo pozo en la región 1 sin pérdidas.')
else:
    print('Es posible desarrollar un nuevo pozo en la región 1 sin pérdidas.')

Volumen medio de reservas para la región 1: 68.82500000000002
No es posible desarrollar un nuevo pozo en la región 1 sin pérdidas.


### Region 2

In [533]:
print('Volumen medio de reservas para la región 2:', target2.mean())

# Compara el valor obtenido con el volumen medio de reservas para la región 2
if sufficient_volume > target2.mean():
    print('No es posible desarrollar un nuevo pozo en la región 2 sin pérdidas.')
else:
    print('Es posible desarrollar un nuevo pozo en la región 2 sin pérdidas.')

Volumen medio de reservas para la región 2: 95.00000000000004
No es posible desarrollar un nuevo pozo en la región 2 sin pérdidas.


### Observaciones
- Ninguna de las tres regiones parece tener el volúmen promedio suficiente para realizar un nuevo pozo sin pérdidas.
- A pesar de que las `regiones 0 y 2` tienen los volúmenes promedio más cercanos a lo necesario para no tener pérdidas, no hay que olvidar que la `región 1` tiene un RECM más bajo.

## Ingreso total esperado de los 200 pozos más rentables de cada región

In [534]:
# Crear función para calcular el ingreso total esperado de los 200 pozos más rentables para una región
def calculate_profit(actual_volumes, predicted_volumes):
    # Ordenar las predicciones de mayor a menor
    sorted_predictions = predicted_volumes.sort_values(ascending=False)

    # Seleccionar los 200 valores más altos
    selected = actual_volumes[sorted_predictions.index][:200]

    # Calcular la ganancia total esperada
    total_revenue = (selected * income_per_unit).sum()
    
    return total_revenue - budget

### Region 0

In [535]:
# Calcular el ingreso total esperado de los 200 pozos más rentables para la región 0
profit0 = calculate_profit(target_valid0, predictions0)
print('Ingreso total esperado de los 200 pozos más rentables para la región 0:', profit0)

Ingreso total esperado de los 200 pozos más rentables para la región 0: 33208260.43139851


### Region 1

In [536]:
# Calcular el ingreso total esperado de los 200 pozos más rentables para la región 1
profit1 = calculate_profit(target_valid1, predictions1)
print('Ingreso total esperado de los 200 pozos más rentables para la región 1:', profit1)

Ingreso total esperado de los 200 pozos más rentables para la región 1: 24150866.966815084


### Region 2

In [537]:
# Calcular el ingreso total esperado de los 200 pozos más rentables para la región 2
profit2 = calculate_profit(target_valid2, predictions2)
print('Ingreso total esperado de los 200 pozos más rentables para la región 2:', profit2)

Ingreso total esperado de los 200 pozos más rentables para la región 2: 25399159.458429456


### Observaciones
- La `región 0` tiene el ingreso total esperado más alto de los tres, seguido por la `región 2` y luego la `región 1`.

## Conclusion intermedia
- Basándose en los resultados del volúmen medio de cada región así como los ingresos esperados aún no se puede decir cuál es la mejor opción de región para la creación de un nuevo pozo.

# Cálculo de riesgos y ganancias para cada región

## Bootstrapping con 1000 muestras para encontrar la distribución del beneficio

In [538]:
# Crear una función para realizar el bootstrapping y calcular la distribución de beneficios y riesgos para cada región
def bootstrap_profit_and_loss(target, predictions, loop_size = 1_000, sample_size = 500):
    # Inicializar la lista de beneficios
    values = []

    # Combinar target y predictions en un DataFrame
    combined = pd.DataFrame({
        'target': target,
        'predictions': predictions
    })

    # Inicializar el contador de beneficios negativos
    negative_profit_count = 0

    # Realizar bootstrapping
    for _ in range(loop_size):
        # Muestrear los datos
        subsample = combined.sample(sample_size, replace=True, random_state=random_state)

        # Separar target y predictions de nuevo
        subsample_target = subsample['target']
        subsample_predictions = subsample['predictions']

        # Calcular el beneficio de la muestra y añadirlo a la lista
        profit = calculate_profit(subsample_target, subsample_predictions)
        values.append(profit)
        
        # Contar el número de veces que el beneficio es negativo
        if profit < 0:
            negative_profit_count += 1

    # Convertir la lista de beneficios a una Serie de pandas
    values = pd.Series(values)

    # Calcular el beneficio promedio
    average_profit = values.mean()

    # Calcular el cuantil del 2.5 %
    lower = values.quantile(risk_threshold)

    # Calcular el cuantil del 97.5 %
    upper = values.quantile(1 - risk_threshold)

    # Evaluar el riesgo de pérdidas
    loss_risk = negative_profit_count / loop_size

    return average_profit, loss_risk, lower, upper

In [539]:
# Crear función para mostrar los resultados del bootstrapping
def print_bootstrap_results(target_valid, predictions, region):
    average_profit, loss_risk, lower, upper = bootstrap_profit_and_loss(target_valid, predictions)

    print(f'Intervalo de confianza del 95% para la región {region}: [{lower:.1f}] -> [{upper:.1f}]')
    print(f'Beneficio promedio para la región {region}: ${average_profit:.1f}')
    print(f'Riesgo de pérdidas para la región {region}: {loss_risk:.1%}')

### Region 0

In [540]:
# Bootstrapping con 1000 muestras para encontrar la distribución del beneficio para la región 0
print_bootstrap_results(target_valid0, predictions0, 0)

Intervalo de confianza del 95% para la región 0: [-761878.1] -> [9578465.3]
Beneficio promedio para la región 0: $4238972.4
Riesgo de pérdidas para la región 0: 4.8%


### Region 1

In [541]:
# Bootstrapping con 1000 muestras para encontrar la distribución del beneficio para la región 1
print_bootstrap_results(target_valid1, predictions1, 1)

Intervalo de confianza del 95% para la región 1: [1080669.0] -> [9285744.4]
Beneficio promedio para la región 1: $5132567.0
Riesgo de pérdidas para la región 1: 0.6%


### Region 2

In [542]:
# Bootstrapping con 1000 muestras para encontrar la distribución del beneficio para la región 2
print_bootstrap_results(target_valid2, predictions2, 2)

Intervalo de confianza del 95% para la región 2: [-1428006.3] -> [8933805.7]
Beneficio promedio para la región 2: $3811203.6
Riesgo de pérdidas para la región 2: 7.4%


## Conclusion intermedia
- Si bien los valores obtenidos a través del bootstrapping pueden variar en cada ejecución del código, hay que considerar que dichos valores se mantienen dentro de un mismo rango.
- La `región 1` muestra en general el `beneficio promedio más alto` así como el `riesgo de pérdidas más bajo`, lo que indica que esta región tiene el mayor potencial para cumplir nuestro objetivo.

# Conclusión general

Durante este proyecto, se construyó un modelo de regresión lineal para predecir el volumen de reservas en nuevos pozos petroleros, basado en tres regiones con datos geológicos dados. Posteriormente, se realizó una simulación de bootstrapping para estimar la distribución de los posibles beneficios y riesgos en cada región.

- Se realizó un análisis exploratorio de los datos y se procedió a entrenar un modelo de regresión lineal para cada una de las tres regiones. Se ajustó un modelo de regresión lineal para cada región, utilizando la métrica RMSE para evaluar el rendimiento del modelo.
- Con base en las predicciones del modelo, se seleccionaron los 200 pozos con las mayores predicciones de volumen de reservas de petróleo para la perforación.
- Para cuantificar la incertidumbre en la decisión de perforación y para estimar el posible beneficio, se empleó una técnica de bootstrapping. Esto proporcionó una distribución de posibles beneficios y el riesgo de pérdidas.
- El análisis de bootstrapping mostró que la `región 1` muestra el `beneficio promedio más alto` así como el `riesgo de pérdidas más bajo`, lo que sugiere que sería la región más prometedora para la exploración de petróleo.

Basándonos en los resultados, se ha concluido que la `región 1` es la más prometedora para futuras operaciones de perforación de pozos, ya que presenta el `beneficio promedio más alto` y el `riesgo de pérdidas más bajo`.
